In [5]:
import os
import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import InceptionResnetV1, MTCNN
from scipy.spatial.distance import cosine
import sqlite3
from datetime import datetime

# SQLite veritabanına bağlan veya oluştur
conn = sqlite3.connect('attendance.db')
cursor = conn.cursor()

# Yoklama tablosunu oluştur
cursor.execute('''
CREATE TABLE IF NOT EXISTS attendance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    timestamp TEXT
)
''')
conn.commit()

In [6]:
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

In [7]:
# Yüz veritabanı
known_faces = {}  # Her kişinin ismi ve yüz özellikleri

# Dataset klasörünü ve kişileri tarayın
dataset_path = 'dataset/'  # Veritabanınızın bulunduğu dizin
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_path):
        embeddings = []  # Bu kişiye ait yüz özelliklerini saklayacağımız liste

        # Kişinin fotoğraflarını oku ve yüz özelliklerini çıkar
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            if image_path.endswith('.jpg'):
                img = Image.open(image_path)
                img_rgb = img.convert('RGB')

                # Yüzleri algıla
                faces = mtcnn(img_rgb)
                if faces is not None:
                    # İlk yüzü al ve özelliklerini çıkar (3D'den 4D'ye dönüştürme)
                    face_tensor = faces[0]  # Yüzü al
                    face_tensor = face_tensor.unsqueeze(0)  # 4D'ye dönüştür (batch size = 1)
                    
                    embedding = model(face_tensor).detach().numpy()
                    embeddings.append(embedding)

        # Kişinin yüz özelliklerini veritabanına ekle
        if embeddings:
            # Ortalama embedding (bütün fotoğraflardan çıkarılan özelliklerin ortalaması)
            known_faces[person_name] = np.mean(embeddings, axis=0)

# Veritabanı hazır!
print("Veritabanı oluşturuldu:", known_faces.keys())

Veritabanı oluşturuldu: dict_keys(['alper_isleyen', 'enes_berk_demirci', 'jenna_ortega', 'robert_downey', 'sardor_abdirayimov', 'serdarikan', 'taylor_swift'])


In [8]:
# Yoklama raporunu görüntüleme fonksiyonu
def show_attendance():
    cursor.execute('SELECT name, timestamp FROM attendance ORDER BY timestamp DESC')
    records = cursor.fetchall()

    print("\nYoklama Listesi:")
    print("Name\t\t\tTimestamp")
    print("-" * 40)
    for name, timestamp in records:
        print(f"{name}\t\t{timestamp}")

# Yüzleri karşılaştırma fonksiyonu
def compare_faces(embedding, known_faces):
    min_distance = float('inf')
    name = None

    # Veritabanındaki yüzlerle karşılaştırma yapalım
    for known_name, known_embedding in known_faces.items():
        # Cosine mesafesini hesaplamak için her iki embedding'i de 1D'ye dönüştürüyoruz
        distance = cosine(embedding.flatten(), known_embedding.flatten())  # flatten() kullanarak 1D yapıyoruz
        if distance < min_distance:
            min_distance = distance
            name = known_name

    # Eşik değeri belirleyelim
    if min_distance < 0.3:  # 0.6, yüzlerin ne kadar benzer olması gerektiğini belirleyen eşik değeri
        return name
    else:
        return "Tanımlanamayan Kişi"
    


# Yoklama kayıt fonksiyonu
recorded_names = set()  # Yoklama alınan isimleri tutacak bir set

def add_attendance(name):
    # Eğer isim zaten kaydedilmişse, tekrar kaydetme
    if name in recorded_names:
        return

    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")

    # Yeni kayıt ekle
    cursor.execute('INSERT INTO attendance (name, timestamp) VALUES (?, ?)', (name, current_time))
    conn.commit()
    recorded_names.add(name)  # İsmi sete ekle
    print(f"{name} için yoklama kaydedildi: {current_time}")


# Kamera görüntüsü
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Kamera açılamadı!")
    exit()

# Kamera akışı üzerinde yüz tanıma
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kare okunamadı!")
            break

        # OpenCV görüntüsünü PIL görüntüsüne dönüştür
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_frame = Image.fromarray(frame_rgb)

        # Yüzleri algılayalım
        boxes, _ = mtcnn.detect(pil_frame)

        # Yüzlerin her birini kontrol edelim
        if boxes is not None:
            draw = ImageDraw.Draw(pil_frame)
            
            for box in boxes:
                # Yüzü dikdörtgenle çizelim
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=3)

                # Yüz özelliklerini çıkaralım
                face = pil_frame.crop((box[0], box[1], box[2], box[3]))
                
                # Yüzü 4D'ye dönüştürme (batch size = 1)
                faces = mtcnn(face)  # MTCNN ile yüzü algılama
                if faces is not None:
                    face_embedding = model(faces[0].unsqueeze(0)).detach().numpy()  # 4D'ye dönüştürme

                    # Veritabanındaki kişilerle karşılaştıralım
                    name = compare_faces(face_embedding, known_faces)
                    font = ImageFont.truetype("arial.ttf", 40)
                    draw.text((box[0], box[1] - 10), name, fill=(255, 0, 0))

                    # Burada yoklama kaydı ekle
                    if name != "Tanımlanamayan Kişi":
                        add_attendance(name)  # Yoklama kaydı ekle

        # PIL'den OpenCV'ye dönüşüm
        frame_with_boxes = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)

        # Görüntüyü gösterelim
        cv2.imshow('Face Recognition', frame_with_boxes)

        # 'q' tuşuna basarak çıkabilirsiniz
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # 'r' tuşuna basıldığında raporu göster
        if cv2.waitKey(1) & 0xFF == ord('r'):
            show_attendance()
finally:
    cap.release()
    cv2.destroyAllWindows()